In [1]:
import torch
import torch.nn as nn


class LeNet(nn.Module):
    def __init__(self):
        super(LeNet, self).__init__()
        # 1 input channel, 6 output channels, 5x5 kernel
        self.conv1 = nn.Conv2d(1, 6, 5)
        # 6 input channels, 6 output channels, 5x5 kernel
        self.pool1 = nn.MaxPool2d(2, 2)

        # 6 input channels, 16 output channels, 5x5 kernel
        self.conv2 = nn.Conv2d(6, 16, 5)
        # 16 input channels, 16 output channels, 5x5 kernel
        self.pool2 = nn.MaxPool2d(2, 2)

        self.fc1 = nn.Linear(16 * 4 * 4, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.conv1(x)  # input shape: (batch_size, 1, 28, 28)
        x = torch.relu(x)  # output shape: (batch_size, 6, 24, 24)
        x = self.pool1(x)  # output shape: (batch_size, 6, 12, 12)

        x = self.conv2(x)  # input shape: (batch_size, 6, 12, 12)
        x = torch.relu(x)  # output shape: (batch_size, 16, 8, 8)
        x = self.pool2(x)  # output shape: (batch_size, 16, 4, 4)

        x = x.view(x.shape[0], -1)  # output shape: (batch_size, 16*4*4)
        x = self.fc1(x)  # output shape: (batch_size, 120)
        x = torch.relu(x)
        x = self.fc2(x)  # output shape: (batch_size, 84)
        x = torch.relu(x)
        x = self.fc3(x)  # output shape: (batch_size, 10)

        return x


def print_params(model):
    count = 0
    for name, param in model.named_parameters():
        count += param.numel()
        print(name, param.shape)
    print('total parameters:', count)


def print_forward(model, x):
    print('input x to model:', x.shape)
    for name, layer in model.named_children():
        if name == 'fc1':
            x = x.view(x.shape[0], -1)
        x = layer(x)
        print(f'{name} after size: {x.shape}')


# model = LeNet()
# print(model)
# print('---')
# print_params(model)

# x = torch.randn(64, 1, 28, 28)
# print_forward(model, x)

LeNet(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=256, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)
---
conv1.weight torch.Size([6, 1, 5, 5])
conv1.bias torch.Size([6])
conv2.weight torch.Size([16, 6, 5, 5])
conv2.bias torch.Size([16])
fc1.weight torch.Size([120, 256])
fc1.bias torch.Size([120])
fc2.weight torch.Size([84, 120])
fc2.bias torch.Size([84])
fc3.weight torch.Size([10, 84])
fc3.bias torch.Size([10])
total parameters: 44426
input x to model: torch.Size([64, 1, 28, 28])
conv1 after size: torch.Size([64, 6, 24, 24])
pool1 after size: torch.Size([64, 6, 12, 12])
conv2 after size: torch.Size([64, 16, 8, 

In [2]:
import torch
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import numpy as np
import random

# 设置全局随机种子
seed = 42
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(seed)
random.seed(seed)


def worker_init_fn(worker_id):
    np.random.seed(seed + worker_id)
    random.seed(seed + worker_id)


transform = transforms.Compose([
    transforms.RandomRotation(12),  # 随机旋转 12 度
    transforms.ToTensor()
])

# 加载训练数据集
train_dataset = datasets.MNIST(
    root='./data', train=True, download=True, transform=transform)


# 定义训练数据加载器
train_loader = DataLoader(
    dataset=train_dataset, batch_size=64, shuffle=True, worker_init_fn=worker_init_fn)

Failed to download (trying next):
HTTP Error 403: Forbidden



100.0%


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100.0%


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100.0%


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100.0%

Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



In [5]:
import wandb
wandb.require("core")

epochs = 10
learning_rate = 0.001
model = LeNet()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
criterion = torch.nn.CrossEntropyLoss()

wandb_init_args = {
    'project': 'MNIST-手写数字识别',
    'config': {
        "learning_rate": learning_rate,
        "architecture": "LeNet-5",
        "dataset": "MNIST",
        "epochs": epochs,
    },
    'reinit': True
}
run = wandb.init(**wandb_init_args, job_type="training")
wandb_init_args['name'] = run.name

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
criterion.to(device)


for epoch in range(epochs):
    model.train()
    for i, (images, labels) in enumerate(train_loader):
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        acc = (outputs.argmax(dim=1) == labels).float().mean()

        if (i + 1) % 100 == 0:
            wandb.log({"loss": loss.item(), "acc": acc.item()})
            print(
                f'Epoch [{epoch + 1}/{epochs}], Step [{i + 1}/{len(train_loader)}], Loss: {loss.item():.4f}')


wandb.finish()

Epoch [1/10], Step [100/938], Loss: 0.5688
Epoch [1/10], Step [200/938], Loss: 0.2963
Epoch [1/10], Step [300/938], Loss: 0.2674
Epoch [1/10], Step [400/938], Loss: 0.3359
Epoch [1/10], Step [500/938], Loss: 0.2410
Epoch [1/10], Step [600/938], Loss: 0.1043
Epoch [1/10], Step [700/938], Loss: 0.1684
Epoch [1/10], Step [800/938], Loss: 0.0776
Epoch [1/10], Step [900/938], Loss: 0.1358
Epoch [2/10], Step [100/938], Loss: 0.2307
Epoch [2/10], Step [200/938], Loss: 0.1969
Epoch [2/10], Step [300/938], Loss: 0.0957
Epoch [2/10], Step [400/938], Loss: 0.0570
Epoch [2/10], Step [500/938], Loss: 0.1891
Epoch [2/10], Step [600/938], Loss: 0.0847
Epoch [2/10], Step [700/938], Loss: 0.0322
Epoch [2/10], Step [800/938], Loss: 0.0514
Epoch [2/10], Step [900/938], Loss: 0.0601
Epoch [3/10], Step [100/938], Loss: 0.0965
Epoch [3/10], Step [200/938], Loss: 0.0859
Epoch [3/10], Step [300/938], Loss: 0.0641
Epoch [3/10], Step [400/938], Loss: 0.0242
Epoch [3/10], Step [500/938], Loss: 0.1934
Epoch [3/10